In [5]:
import os
import requests
import geojson
import pandas as pd
import geopandas as gpd
from shapely.geometry import shape
from dotenv import load_dotenv

# Load API key from .env file
load_dotenv('/Users/ronaldsheaks/Desktop/OSU AI Bootcamp/Group Project/GitHub Repo/OSU-AI-Bootcamp-Group-Project-1-Group-4/API Keys/.env')
mapbox_access_token = os.getenv('MapBox_API_Key')

# Read the Whole Foods locations CSV
csv_path = '/Users/ronaldsheaks/Desktop/OSU AI Bootcamp/Group Project/GitHub Repo/OSU-AI-Bootcamp-Group-Project-1-Group-4/Resources/Ohio_Whole_Foods_Locations.csv'
locations_df = pd.read_csv(csv_path)

# Function to get isochrone from Mapbox API
def get_isochrone(lat, lon, minutes=20):
    url = f"https://api.mapbox.com/isochrone/v1/mapbox/driving/{lon},{lat}?contours_minutes={minutes}&polygons=true&access_token={mapbox_access_token}"
    response = requests.get(url)
    return response.json()

# List to hold isochrone polygons
isochrones = []

# Generate isochrones for each Whole Foods location
for index, row in locations_df.iterrows():
    lat, lon = row['latitude'], row['longitude']
    isochrone = get_isochrone(lat, lon)
    for feature in isochrone['features']:
        isochrones.append(shape(feature['geometry']))

# Create a GeoDataFrame with the isochrones
gdf = gpd.GeoDataFrame(geometry=isochrones)

# Save the GeoDataFrame as a Shapefile
shapefile_path = '/Users/ronaldsheaks/Desktop/OSU AI Bootcamp/Group Project/GitHub Repo/OSU-AI-Bootcamp-Group-Project-1-Group-4/Resources/Shapefiles/WF_20_Min_DT_Ohio_Mapbox/WF_20_Min_DT_Ohio_Mapbox.shp'
gdf.to_file(shapefile_path)

print(f"Isochrone polygons saved as {shapefile_path}")


Isochrone polygons saved as /Users/ronaldsheaks/Desktop/OSU AI Bootcamp/Group Project/GitHub Repo/OSU-AI-Bootcamp-Group-Project-1-Group-4/Resources/Shapefiles/WF_20_Min_DT_Ohio_Mapbox/WF_20_Min_DT_Ohio_Mapbox.shp


/opt/anaconda3/lib/python3.11/site-packages/pyogrio/raw.py:698: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  warnings.warn(
